In [ ]:
# === セル1: Google Drive のマウント & ファイルパス設定 ===
from google.colab import drive
# 初回のみ実行後、ブラウザの認証に従ってログインしてください
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# txt ファイルは、Google Drive の下記パスに配置してください。
# 例: /MyDrive/pizza_recipes/artichoke_pizza.txt
FILE_PATH = '/content/drive/MyDrive/pizza_recipes/artichoke_pizza.txt'
print(f"Reading recipe from: {FILE_PATH}")

Reading recipe from: /content/drive/MyDrive/pizza_recipes/artichoke_pizza.txt


In [ ]:
# === セル2: ライブラリ読み込み & OpenAI クライアント初期化 ===
import json
import re
from getpass import getpass
from openai import OpenAI

# 安全のため、Notebook 実行時にキーを入力する方式を推奨します
api_key = getpass('Enter your OpenAI API key: ')
client = OpenAI(api_key=api_key)

Enter your OpenAI API key: ··········


In [ ]:
# === セル3: レシピデータ読み込み ===
with open(FILE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

print("Title:", data.get("title"))
print("Number of steps:", len(data.get("table", [])))

Title: artichoke_pizza
Number of steps: 4


In [ ]:
# === セル4: Chain-of-Thought プロンプト実行関数 (NA対応 & 逐次連携 & 出力マッピング) ===
def run_cot_step(instruction, action, prev_out='NA'):
    # 前ステップ出力の追記
    prev_line = f"Previous output: {prev_out}\n" if prev_out != 'NA' else ''
    # プロンプト構築
    user_prompt = (
        f"Instruction: {instruction}\n"
        f"Action: {action}\n"
        f"{prev_line}"
        "Think step by step.\n"
        "If the action is 'preheat', produce exactly:\n"
        "<in>: NA</in>\n"
        "<out>: NA</out>\n"
        "Otherwise, build the <in> tag as follows:\n"
        "- If previous output is not NA, include it first.\n"
        "- Then list only the ingredients explicitly needed.\n"
        "Map the final output state to these phrases:\n"
        "- mixing flour and water -> dough\n"
        "- spreading tomato sauce -> sauced dough\n"
        "- adding toppings (e.g., mushrooms, artichokes, olives) -> topped dough\n"
        "- baking the pizza -> baked pizza\n"
        "Output ONLY in this exact format:\n"
        "<in>: (item1; item2; ...)</in>\n"
        "<out>: result_phrase</out>"
    )
    messages = [
        {"role": "system", "content": "You are a precise cooking assistant. Respond only with <in> and <out> tags."},
        {"role": "user", "content": user_prompt}
    ]
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.0,
        max_tokens=150
    )
    return resp.choices[0].message.content.strip()

In [ ]:
# === セル5: 全ステップに適用して結果取得 & 表示 ===
results = []
for i, row in enumerate(data.get("table", []), start=1):
    inst = row.get("instructions")
    act  = row.get("actions")
    print(f"\n=== Step {i}: {inst} ===")
    cot_output = run_cot_step(inst, act)
    print(cot_output)
    results.append({"step": i, "cot_output": cot_output})


=== Step 1: heat up your oven as hot as it gets . ===
<in>: NA</in>
<out>: NA</out>

=== Step 2: spread tomato_sauce evenly to the both pizzas . ===
<in>: dough; tomato_sauce</in>
<out>: sauced dough</out>

=== Step 3: spread the mushrooms , artichokes and olives on the pizza . ===
<in>: mushrooms; artichokes; olives</in>
<out>: topped dough</out>

=== Step 4: bake in the oven for about 10 minutes or longer , depending on how hot your oven is . ===
<in>: topped dough</in>
<out>: baked pizza</out>


In [ ]:
# === セル6: <in> / <out> の自動抽出（正規表現更新） ===
# クローズタグ </in> または誤記ケース <\in> に対応
print("\n=== Parsed <in> and <out> from CoT outputs ===")
for r in results:
    text = r['cot_output']
    # <in>タグの抽出: </in> or <\in>
    in_m  = re.search(r"<in>\s*(.*?)\s*(?:</in>|<\\in>)", text)
    out_m = re.search(r"<out>\s*(.*?)\s*(?:</out>|<\\out>)", text)
    inp   = in_m.group(1).strip() if in_m else ''
    outp  = out_m.group(1).strip() if out_m else ''
    print(f"Step {r['step']}  Input: {inp}\n           Output: {outp}\n")


=== Parsed <in> and <out> from CoT outputs ===
Step 1  Input: : NA
           Output: : NA

Step 2  Input: : dough; tomato_sauce
           Output: : sauced dough

Step 3  Input: : mushrooms; artichokes; olives
           Output: : topped dough

Step 4  Input: : topped dough
           Output: : baked pizza

